In [ ]:
#importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import re
import seaborn as sns

In [ ]:
'''
OBJETIVOS DA SEMANA:
- Analisar a mudança dos grupos de acordo com o K
- Analisar atraves de 2 testes a porcentagem de mudança dos grupos
- Analisar a consistencia dos grupos
'''

In [ ]:
# Pasta contendo os arquivos JSON
pasta_keywords = 'keywords_k'

# Lista para armazenar os DataFrames de cada arquivo
dataframes = []

# Itera sobre os arquivos na pasta
for nome_arquivo in os.listdir(pasta_keywords):
    caminho_arquivo = os.path.join(pasta_keywords, nome_arquivo)

    try:
        # Extrai o valor de k do nome do arquivo
        match = re.search(r'keywords_k=(\d+)', nome_arquivo)
        if match:
            k_value = int(match.group(1))
        else:
            k_value = None

        # Leitura do arquivo JSON com o encoding utf-8-sig
        with open(caminho_arquivo, 'r', encoding='utf-8-sig') as f:
            data = json.load(f)

        # Extrai a lista de itens sob a chave 'items'
        items = data.get('network', {}).get('items', [])

        # Converte para DataFrame
        df = pd.json_normalize(items)

        # Adiciona uma coluna 'k' com o valor extraído do nome do arquivo
        df['k'] = k_value

        # Adiciona o DataFrame à lista
        dataframes.append(df)

        # Conta o número de clusters únicos para cada valor de k
        num_clusters = df['cluster'].nunique()

    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar o JSON em {caminho_arquivo}: {e}")
    except Exception as e:
        print(f"Erro desconhecido em {caminho_arquivo}: {e}")

# Concatena todos os DataFrames em um único DataFrame
df_completo = pd.concat(dataframes, ignore_index=True)

#Ordena o DataFrame por k
df_completo.sort_values(by='k', inplace=True)

# imprimir a cabeça do dataframe
df_completo.head()

In [ ]:
# Vamos analisar apenas os dados com k = 9 e k = 10
df_910 = df_completo[(df_completo['k'] == 9) | (df_completo['k'] == 10)]

# Criação do dicionário para armazenar informações sobre os clusters para k=9 e k=10
cluster_info = {}

# Itera sobre os labels únicos no DataFrame df_910
for label in df_910['label'].unique():
    # Filtra o DataFrame para o label específico
    label_df = df_910[df_910['label'] == label]
    
    # Obtém o cluster para k=9, se existir
    cluster_k9 = label_df[label_df['k'] == 9]['cluster'].values[0] if label_df[label_df['k'] == 9].shape[0] > 0 else None
    
    # Obtém o cluster para k=10, se existir
    cluster_k10 = label_df[label_df['k'] == 10]['cluster'].values[0] if label_df[label_df['k'] == 10].shape[0] > 0 else None
    
    # Adiciona as informações ao dicionário
    cluster_info[label] = {'k=9': cluster_k9, 'k=10': cluster_k10}

# Imprime o dicionário
print("Dicionário de informações sobre clusters para k=9 e k=10:")
print(cluster_info)

In [ ]:
#imprimir quantos labels temos no dicionario
print(f"Quantidade de labels: {len(cluster_info)}")

#imprimir quantos labels temos para k=9 e k=10 em df_910
print(f"Quantidade de labels com k=9: {df_910[df_910['k'] == 9]['label'].nunique()}")
print(f"Quantidade de labels com k=10: {df_910[df_910['k'] == 10]['label'].nunique()}")

#transformar o dicionario em um dataframe
df_cluster_info = pd.DataFrame.from_dict(cluster_info, orient='index')

#ordenar o dataframe por k=9 e k=10
df_cluster_info.sort_values(by=['k=9', 'k=10'], inplace=True)

df_cluster_info

In [ ]:
#criar uma matriz de cluster para k=9 e k=10, com 0
cluster_matrix = np.zeros((len(df_cluster_info), len(df_cluster_info)))

#mostrar a matriz
cluster_matrix